In [1]:
import pandas as pd
from tls_paper_funcs import tls_12_appdata_filtering, remove_empty
from tls_paper_funcs import handshake_removal_only, cert_size_inference_simple_diagram
import numpy as np

from tqdm.notebook import tqdm
from multiprocess import Pool

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
metasploit_folder = 'data/ms/'
metasploit_variants = [a+'-'+b for a in ['tls12', 'tls13'] for b in ['shortcert', 'longcert']]
dfs = []
for variant in metasploit_variants:
    df = pd.read_csv(metasploit_folder + variant + '/all.csv')
    df['tlsver'] = variant.split('-')[0]
    df['certver'] = variant.split('-')[1]
    dfs.append(df)
full_df_ms = pd.concat(dfs, sort=False, ignore_index=True)
full_df_ms.replace(-2,-1, inplace=True) # joy processing mistake

full_df_ms

,c_ip,c_port,s_ip,s_port,ip_proto,start_time,ip_b_0,ip_b_1,ip_b_2,ip_b_3,ip_b_4,ip_b_5,ip_b_6,ip_b_7,ip_b_8,ip_b_9,ip_b_10,ip_b_11,ip_b_12,ip_b_13,ip_b_14,ip_b_15,ip_b_16,ip_b_17,ip_b_18,ip_b_19,ip_ipt_0,ip_ipt_1,ip_ipt_2,ip_ipt_3,ip_ipt_4,ip_ipt_5,ip_ipt_6,ip_ipt_7,ip_ipt_8,ip_ipt_9,ip_ipt_10,ip_ipt_11,ip_ipt_12,ip_ipt_13,ip_ipt_14,ip_ipt_15,ip_ipt_16,ip_ipt_17,ip_ipt_18,ip_ipt_19,ip_dir_0,ip_dir_1,ip_dir_2,ip_dir_3,ip_dir_4,ip_dir_5,ip_dir_6,ip_dir_7,ip_dir_8,ip_dir_9,ip_dir_10,ip_dir_11,ip_dir_12,ip_dir_13,ip_dir_14,ip_dir_15,ip_dir_16,ip_dir_17,ip_dir_18,ip_dir_19,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_ipt_0,tls_ipt_1,tls_ipt_2,tls_ipt_3,tls_ipt_4,tls_ipt_5,tls_ipt_6,tls_ipt_7,tls_ipt_8,tls_ipt_9,tls_ipt_10,tls_ipt_11,tls_ipt_12,tls_ipt_13,tls_ipt_14,tls_ipt_15,tls_ipt_16,tls_ipt_17,tls_ipt_18,tls_ipt_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,tls_tp_0,tls_tp_1,tls_tp_2,tls_tp_3,tls_tp_4,tls_tp_5,tls_tp_6,tls_tp_7,tls_tp_8,tls_tp_9,tls_tp_10,tls_tp_11,tls_tp_12,tls_tp_13,tls_tp_14,tls_tp_15,tls_tp_16,tls_tp_17,tls_tp_18,tls_tp_19,command,tlsver,certver
0,172.18.0.2,51236,172.18.0.3,443,6,1.649326e+09,188,1346,93,242,417,195,31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,43,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,183,65,957,300,4,37,1,40,186,1,40,412,190,26,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,43,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,22,20,22,23,23,21,-1,-1,-1,-1,-1,-1,route,tls12,shortcert
1,172.18.0.2,51238,172.18.0.3,443,6,1.649326e+09,188,1346,93,242,268,102,31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,11,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,183,65,957,300,4,37,1,40,186,1,40,263,97,26,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,11,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,22,20,22,23,23,21,-1,-1,-1,-1,-1,-1,route,tls12,shortcert
2,172.18.0.2,51240,172.18.0.3,443,6,1.649326e+09,188,1346,93,242,268,142,31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,183,65,957,300,4,37,1,40,186,1,40,263,137,26,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,22,20,22,23,23,21,-1,-1,-1,-1,-1,-1,route,tls12,shortcert
3,172.18.0.2,51242,172.18.0.3,443,6,1.649326e+09,188,1346,93,242,268,142,31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,183,65,957,300,4,37,1,40,186,1,40,263,137,26,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,22,20,22,23,23,21,-1,-1,-1,-1,-1,-1,route,tls12,shortcert
4,172.18.0.2,51244,172.18.0.3,443,6,1.649326e+09,188,1346,93,242,268,531,31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,183,65,957,300,4,37,1,40,186,1,40,263,526,26,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,22,20,22,23,23,21,-1,-1,-1,-1,-1,-1,route,tls12,shortcert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [3]:
full_df_ms['tls_dir_19'].value_counts()

tls_dir_19
-1    782226
 1      9000
Name: count, dtype: int64

In [4]:
782226 / 791226

0.988625247400869

In [5]:
#full_df_ms.to_csv('data/ms/ms-full.csv', index=False)

In [6]:
ms_certs_length = pd.read_csv(metasploit_folder + 'certificates_length.csv')
ms_certs_length

,frame.time_epoch,ip.src,tcp.srcport,ip.dst,tcp.dstport,tls.handshake.certificates_length,tlsver,certver,command
0,1.649254e+09,172.18.0.3,443,172.18.0.2,57008,2547,tls12,longcert,multiple
1,1.649254e+09,172.18.0.3,443,172.18.0.2,57010,2547,tls12,longcert,multiple
2,1.649254e+09,172.18.0.3,443,172.18.0.2,57012,2547,tls12,longcert,multiple
3,1.649254e+09,172.18.0.3,443,172.18.0.2,57014,2547,tls12,longcert,multiple
4,1.649254e+09,172.18.0.3,443,172.18.0.2,57016,2547,tls12,longcert,multiple
...,...,...,...,...,...,...,...,...,...
2373673,1.649341e+09,172.18.0.3,443,172.18.0.2,54858,1004,tls13,shortcert,sysinfo
2373674,1.649341e+09,172.18.0.3,443,172.18.0.2,54860,1004,tls13,shortcert,sysinfo
2373675,1.649341e+09,172.18.0.3,443,172.18.0.2,54862,1004,tls13,shortcert,sysinfo
2373676,1.649341e+09,172.18.0.3,443,172.18.0.2,54864,1004,tls13,shortcert,sysinfo


In [7]:
display(ms_certs_length[ms_certs_length.duplicated()])
ms_certs_length.drop_duplicates(ignore_index=True, inplace=True)
ms_certs_length

,frame.time_epoch,ip.src,tcp.srcport,ip.dst,tcp.dstport,tls.handshake.certificates_length,tlsver,certver,command
791226,1.649254e+09,172.18.0.3,443,172.18.0.2,57008,2547,tls12,longcert,multiple
791227,1.649254e+09,172.18.0.3,443,172.18.0.2,57010,2547,tls12,longcert,multiple
791228,1.649254e+09,172.18.0.3,443,172.18.0.2,57012,2547,tls12,longcert,multiple
791229,1.649254e+09,172.18.0.3,443,172.18.0.2,57014,2547,tls12,longcert,multiple
791230,1.649254e+09,172.18.0.3,443,172.18.0.2,57016,2547,tls12,longcert,multiple
...,...,...,...,...,...,...,...,...,...
2373673,1.649341e+09,172.18.0.3,443,172.18.0.2,54858,1004,tls13,shortcert,sysinfo
2373674,1.649341e+09,172.18.0.3,443,172.18.0.2,54860,1004,tls13,shortcert,sysinfo
2373675,1.649341e+09,172.18.0.3,443,172.18.0.2,54862,1004,tls13,shortcert,sysinfo
2373676,1.649341e+09,172.18.0.3,443,172.18.0.2,54864,1004,tls13,shortcert,sysinfo


,frame.time_epoch,ip.src,tcp.srcport,ip.dst,tcp.dstport,tls.handshake.certificates_length,tlsver,certver,command
0,1.649254e+09,172.18.0.3,443,172.18.0.2,57008,2547,tls12,longcert,multiple
1,1.649254e+09,172.18.0.3,443,172.18.0.2,57010,2547,tls12,longcert,multiple
2,1.649254e+09,172.18.0.3,443,172.18.0.2,57012,2547,tls12,longcert,multiple
3,1.649254e+09,172.18.0.3,443,172.18.0.2,57014,2547,tls12,longcert,multiple
4,1.649254e+09,172.18.0.3,443,172.18.0.2,57016,2547,tls12,longcert,multiple
...,...,...,...,...,...,...,...,...,...
791221,1.649341e+09,172.18.0.3,443,172.18.0.2,54858,1004,tls13,shortcert,sysinfo
791222,1.649341e+09,172.18.0.3,443,172.18.0.2,54860,1004,tls13,shortcert,sysinfo
791223,1.649341e+09,172.18.0.3,443,172.18.0.2,54862,1004,tls13,shortcert,sysinfo
791224,1.649341e+09,172.18.0.3,443,172.18.0.2,54864,1004,tls13,shortcert,sysinfo


In [8]:
ms_tlsdata = tls_12_appdata_filtering(full_df_ms[full_df_ms['tlsver'] == 'tls12'])
ms_tlsdata['command'] = full_df_ms[full_df_ms['tlsver'] == 'tls12']['command']
ms_tlsdata['certver'] = full_df_ms[full_df_ms['tlsver'] == 'tls12']['certver']
ms_tlsdata

,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,command,certver
0,412.0,190.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,route,shortcert
1,263.0,97.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,route,shortcert
2,263.0,137.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,route,shortcert
3,263.0,137.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,route,shortcert
4,263.0,526.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,route,shortcert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396320,345.0,472.0,137.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,ps,longcert
396321,323.0,251.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,ps,longcert
396322,345.0,472.0,137.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,ps,longcert
396323,323.0,251.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,ps,longcert


In [9]:
#ms_tlsdata.to_csv('processed-datasets/ms-tls12-data.csv')

In [10]:
## tls1.3:
ms13_filtered = tls_12_appdata_filtering( full_df_ms[(full_df_ms['tlsver'] == 'tls13')] ).astype(int)
ms13_filtered.index = full_df_ms[(full_df_ms['tlsver'] == 'tls13')].index
ms13_filtered['certver'] = full_df_ms[(full_df_ms['tlsver'] == 'tls13')]['certver']
ms13_filtered['command'] = full_df_ms[(full_df_ms['tlsver'] == 'tls13')]['command']
ms13_filtered

,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,certver,command
396325,23,975,281,69,69,250,250,312,230,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,1,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,shortcert,route
396326,23,975,281,69,69,302,250,250,90,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,shortcert,route
396327,23,975,281,69,69,302,250,250,130,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,shortcert,route
396328,23,975,281,69,69,302,250,250,519,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,shortcert,route
396329,23,975,281,69,69,324,414,250,250,130,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,shortcert,route
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791221,23,2590,281,69,69,307,250,250,244,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,longcert,ps
791222,23,2590,281,69,69,329,250,465,250,130,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,longcert,ps
791223,23,2590,281,69,69,250,307,250,130,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,1,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,longcert,ps
791224,23,2590,281,69,69,307,250,250,244,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,longcert,ps


In [11]:
remove_empty(ms13_filtered, True)

0


In [12]:
#ms13_filtered.to_csv('processed-datasets/ms-tls13-data.csv')

In [13]:
def tls_hs_remove_parallel(i):
    nst_discard_b, nst_discard_dir = handshake_removal_only(ms13_filtered.loc[i])
    line = np.hstack( [nst_discard_b,   (20 - len(nst_discard_b))*[-1],
                       nst_discard_dir, (20 - len(nst_discard_dir))*[-1]] )
    return (i,line)

In [14]:
tls13samples = list(ms13_filtered.index)
with Pool() as p:
    tls13_appdata_no_hs = list(
        tqdm(
            p.imap(tls_hs_remove_parallel, tls13samples),
            total=len(tls13samples)
        )
    )

  0%|          | 0/394901 [00:00<?, ?it/s]

In [15]:
tls_bs = ['tls_b_'+str(x) for x in range(20)]
tls_dir = ['tls_dir_'+str(x) for x in range(20)]
ms_tlsdata13 = pd.DataFrame(np.vstack([b for a,b in tls13_appdata_no_hs]),
                            columns= tls_bs+tls_dir,
                            index=[a for a,b in tls13_appdata_no_hs])
to_remove13 = ms_tlsdata13[ (ms_tlsdata13[tls_bs] == -1).all(axis=1) ].index
print(len(to_remove13))

1


In [16]:
ms_tlsdata13['certver'] = full_df_ms[(full_df_ms['tlsver'] == 'tls13')]['certver']
ms_tlsdata13['command'] = full_df_ms[(full_df_ms['tlsver'] == 'tls13')]['command']

ms_tlsdata13.drop(to_remove13, inplace=True)
ms_tlsdata13

,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,certver,command
396325,312.0,230.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,shortcert,route
396326,302.0,90.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,shortcert,route
396327,302.0,130.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,shortcert,route
396328,302.0,519.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,shortcert,route
396329,324.0,414.0,130.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,shortcert,route
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791221,307.0,244.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,longcert,ps
791222,329.0,250.0,465.0,250.0,130.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,longcert,ps
791223,307.0,130.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,longcert,ps
791224,307.0,244.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,longcert,ps


In [17]:
#ms_tlsdata13.to_csv('processed-datasets/ms-tls13-behav-new.csv')

In [18]:
# Cert Size Inference
ms_tls13 = full_df_ms[full_df_ms['tlsver'] == 'tls13'].copy().reset_index(drop=True)
ms_tls13

,c_ip,c_port,s_ip,s_port,ip_proto,start_time,ip_b_0,ip_b_1,ip_b_2,ip_b_3,ip_b_4,ip_b_5,ip_b_6,ip_b_7,ip_b_8,ip_b_9,ip_b_10,ip_b_11,ip_b_12,ip_b_13,ip_b_14,ip_b_15,ip_b_16,ip_b_17,ip_b_18,ip_b_19,ip_ipt_0,ip_ipt_1,ip_ipt_2,ip_ipt_3,ip_ipt_4,ip_ipt_5,ip_ipt_6,ip_ipt_7,ip_ipt_8,ip_ipt_9,ip_ipt_10,ip_ipt_11,ip_ipt_12,ip_ipt_13,ip_ipt_14,ip_ipt_15,ip_ipt_16,ip_ipt_17,ip_ipt_18,ip_ipt_19,ip_dir_0,ip_dir_1,ip_dir_2,ip_dir_3,ip_dir_4,ip_dir_5,ip_dir_6,ip_dir_7,ip_dir_8,ip_dir_9,ip_dir_10,ip_dir_11,ip_dir_12,ip_dir_13,ip_dir_14,ip_dir_15,ip_dir_16,ip_dir_17,ip_dir_18,ip_dir_19,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_ipt_0,tls_ipt_1,tls_ipt_2,tls_ipt_3,tls_ipt_4,tls_ipt_5,tls_ipt_6,tls_ipt_7,tls_ipt_8,tls_ipt_9,tls_ipt_10,tls_ipt_11,tls_ipt_12,tls_ipt_13,tls_ipt_14,tls_ipt_15,tls_ipt_16,tls_ipt_17,tls_ipt_18,tls_ipt_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,tls_tp_0,tls_tp_1,tls_tp_2,tls_tp_3,tls_tp_4,tls_tp_5,tls_tp_6,tls_tp_7,tls_tp_8,tls_tp_9,tls_tp_10,tls_tp_11,tls_tp_12,tls_tp_13,tls_tp_14,tls_tp_15,tls_tp_16,tls_tp_17,tls_tp_18,tls_tp_19,command,tlsver,certver
0,172.18.0.2,40176,172.18.0.3,443,6,1.649343e+09,517,1495,80,255,255,317,235,24,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,0,7,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,1,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,512,122,23,975,281,69,1,69,250,250,312,230,19,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,7,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,1,0,0,1,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,23,23,23,23,20,23,23,23,23,23,23,-1,-1,-1,-1,-1,-1,-1,route,tls13,shortcert
1,172.18.0.2,40178,172.18.0.3,443,6,1.649343e+09,517,1495,80,307,255,255,95,24,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,12,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,512,122,23,975,281,69,1,69,302,250,250,90,19,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,12,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,1,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,23,23,23,23,20,23,23,23,23,23,23,-1,-1,-1,-1,-1,-1,-1,route,tls13,shortcert
2,172.18.0.2,40180,172.18.0.3,443,6,1.649343e+09,517,1495,80,307,255,255,135,24,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,512,122,23,975,281,69,1,69,302,250,250,130,19,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,1,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,23,23,23,23,20,23,23,23,23,23,23,-1,-1,-1,-1,-1,-1,-1,route,tls13,shortcert
3,172.18.0.2,40182,172.18.0.3,443,6,1.649343e+09,517,1495,80,307,255,255,524,24,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,512,122,23,975,281,69,1,69,302,250,250,519,19,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,1,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,23,23,23,23,20,23,23,23,23,23,23,-1,-1,-1,-1,-1,-1,-1,route,tls13,shortcert
4,172.18.0.2,40184,172.18.0.3,443,6,1.649343e+09,517,1495,80,329,419,255,255,135,24,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,512,122,23,975,281,69,1,69,324,414,250,250,130,19,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,1,1,1,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,22,22,23,23,23,23,20,23,23,23,23,23,23,23,-1,-1,-1,-1,-1,-1,route,tls13,shortcert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [19]:
def cert_size_guess_parallel(row_id):
    return cert_size_inference_simple_diagram(ms_tls13.loc[row_id], row_id), row_id

with Pool() as p:
    pool_outputs = list(
        tqdm(
            p.imap(cert_size_guess_parallel, range(len(ms_tls13))),
            total=len(ms_tls13)
        )
    )

  0%|          | 0/394901 [00:00<?, ?it/s]

In [20]:
for cert_guess, row_id in pool_outputs:
    ms_tls13.loc[row_id, 'cert_size_guess_simple'] = cert_guess

ms_tls13['cert_size_guess_simple'].describe()

count    394901.000000
mean       2036.771444
std         730.847658
min         944.000000
25%        1046.000000
50%        2544.000000
75%        2557.000000
max        2579.000000
Name: cert_size_guess_simple, dtype: float64

In [21]:
#ms_tls13[['command', 'tlsver', 'certver', 'cert_size_guess_simple']].to_csv('processed-datasets/ms-tls13-cert-inf.csv')